In [1]:
from utils import *

In [2]:
%load_ext autoreload
%autoreload 2

#### Load data

In [3]:
xlsx_pth = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/manus/recorded_data.xlsx"
data = Data(xlsx_pth)
positions_FT = data.getFingertips()

#### Get real data

In [4]:
bone = 'LeftSecondFT'
frames = [5900, 6009, 6190, 6340, 6423, 7690]

touch_locations_real = get_touch_locations_real(positions_FT, bone, frames)

#### Get simulated data

In [5]:
mesh_pth = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/estimate_object_pose/ycb_converted/035_power_drill.ply"
picked_ids = [6796, 5958, 3626, 153, 563, 4257]
touch_locations_simu, pcd = get_touch_locations_simu(mesh_pth, picked_ids=picked_ids, scale=1.)

#### Translate and rotate real points

In [6]:
Q = np.array(touch_locations_simu).T
P = np.array(touch_locations_real).T

R, t = rigid_transform_3D(P, Q)

#plot_bar_values(norm_values)

In [7]:
red = [225/255, 87/255, 89/255]
orange = [242/255, 142/255, 43/255]
yellow = [237/255, 201/255, 72/255]
blue = [78/255, 121/255, 167/255]
green = [89/255, 161/255, 79/255]

In [8]:
touch_locations_real_tr = (R.dot(touch_locations_real.T) + t).T

balls_real_tr = get_balls(touch_locations_real_tr, color=[0,1,0])
balls_simu = get_balls(touch_locations_simu)

In [9]:
line_set = get_lines(touch_locations_real_tr, touch_locations_simu)

visualize = [pcd] + balls_real_tr + balls_simu + [line_set]

o3d.visualization.draw_geometries(visualize, zoom=0.9,
                                  front=[0.0, 0.0, -0.5],
                                  lookat=[0.0, 0.0, 0.0],
                                  up=[0.0, 0.4, 0.0])

#### Translate and rotate simulated points

In [10]:
Q = np.array(touch_locations_real).T
P = np.array(touch_locations_simu).T 

R, t = rigid_transform_3D(P, Q)

In [11]:
import copy

touch_locations_simu_tr = (R.dot(touch_locations_simu.T) + t).T

balls_real = get_balls(touch_locations_real)
balls_simu_tr = get_balls(touch_locations_simu_tr, color=[0,1,0])

line_set = get_lines(touch_locations_real, touch_locations_simu_tr)

pcd_tr = copy.deepcopy(pcd)
pcd_tr.rotate(R, center=(0, 0, 0))
pcd_tr.translate((t))

visualize = [pcd_tr] + balls_real + balls_simu_tr + [line_set]

o3d.visualization.draw_geometries(visualize)

#### Save keypoints for MANO

In [12]:
positions_order_l = [
    "LeftCarpus",
    "LeftFirstMC",
    "LeftFirstPP",
    "LeftFirstDP",
    "LeftFirstFT",
    
    "LeftSecondPP",
    "LeftSecondMP",
    "LeftSecondDP",
    "LeftSecondFT",
    
    "LeftThirdPP",
    "LeftThirdMP",
    "LeftThirdDP",
    "LeftThirdFT",
    
    "LeftFourthPP",
    "LeftFourthMP",
    "LeftFourthDP",
    "LeftFourthFT", 
    
    "LeftFifthPP",
    "LeftFifthMP",
    "LeftFifthDP",
    "LeftFifthFT"     
]

positions_order_r = [position.replace("Left", "Right") for position in positions_order_l]

In [55]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

frame = 0

fig = plt.figure()
ax = fig.gca(projection='3d')

for i, a in enumerate(positions_order_l):
    x, y, z = positions_FT.loc[frame, a]
    
    ax.scatter(xs=x, ys=y, zs=z)
    ax.text(x, y, z, str(i), zdir=None)
    
set_axes_equal(ax)
plt.axis('off')
ax.view_init(azim=110, elev=168)


In [52]:
frame = 0

keypoints_l = np.zeros((len(positions_order_l), 3))

#zero = 'LeftThirdMC'
zero = 'LeftThirdPP'

position_zero = positions_FT.loc[frame, zero] * 1000 

for i, position in enumerate(positions_order_l):
    keypoints_l[i,:] = positions_FT.loc[frame, position] * 1000 - position_zero

In [53]:
keypoints_l

array([[-32.416     , -16.316     ,  72.905     ],
       [ -2.678     ,  -2.527     ,  62.032     ],
       [ 25.011     ,  -3.998     ,  47.156     ],
       [ 48.721     ,  -5.526     ,  33.072     ],
       [ 72.15744555,  -7.15706095,  18.5483276 ],
       [ 17.138     ,   7.59      ,  10.708     ],
       [ 35.101     ,   8.937     , -21.344     ],
       [ 44.947     ,   8.972     , -38.834     ],
       [ 53.55172243,  11.99032584, -54.38576019],
       [  0.        ,   0.        ,   0.        ],
       [ 18.513     ,  -4.801     , -35.394     ],
       [ 29.986     ,  -8.737     , -55.872     ],
       [ 37.0945552 ,  -9.49074462, -73.38529118],
       [-18.535     ,  -8.335     ,  -5.543     ],
       [ -1.071     ,  -7.758     , -39.499     ],
       [  9.232     ,  -8.882     , -59.635     ],
       [ 17.13270909,  -5.51386881, -76.68830831],
       [-37.004     , -16.768     ,  -5.027     ],
       [-22.507     , -18.567     , -33.366     ],
       [-14.292     , -20.703  

In [54]:
frame = 0

keypoints_r = np.zeros((len(positions_order_r), 3))

#zero = 'RightThirdMC'
zero = 'RightThirdPP'

position_zero = positions_FT.loc[frame, zero] * 1000 

for i, position in enumerate(positions_order_r):
    keypoints_r[i,:] = positions_FT.loc[frame, position] * 1000 - position_zero

### Plot mano position

In [95]:
lines = [(0,1), (1,2), (2,3), (3,4), 
         (0,5), (5,6), (6,7), (7,8), 
         (0,9), (9,10), (10,11), (11, 12), 
         (0,13), (13, 14), (14, 15), (15,16), 
         (0,17), (17,18), (18,19), (19,20)]



In [96]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

frame = 3463

fig = plt.figure()
ax = fig.gca(projection='3d')

for i, a in enumerate(positions_order_l):
    
    color = 'C' + str((i-1)//4 + 1) 
    
    x, y, z = positions_FT.loc[frame, a]
    
    ax.scatter(xs=x, ys=y, zs=z, color=color)
    #ax.text(x, y, z, str(i), zdir=None)

for start, end in lines:
    start_pos = positions_order_l[start]
    end_pos = positions_order_l[end]
    xyz = np.vstack([positions_FT.loc[frame, start_pos], positions_FT.loc[frame, end_pos]])
    ax.plot(xyz[:,0], xyz[:,1], xyz[:,2], c='black')
    
set_axes_equal(ax)
plt.axis('off')
ax.view_init(azim=110, elev=168)


In [91]:
xyz = np.vstack([positions_FT.loc[frame, start_pos], positions_FT.loc[frame, end_pos]])

In [92]:
xyz[:,0]

array([-0.282438  , -0.29589237])

# Rotate keypoints_l

In [35]:
A = keypoints_l[[0,5,9,13,17]].T

B = np.array([[-94.6605,   1.4790,   3.3575],
             [ -6.5632,  -3.7214,  24.0435],
             [  0.0000,   0.0000,   0.0000],
             [-12.9249,  -2.4785, -23.3157],
             [-25.8735,  -8.4614, -39.8518]]).T

R, t = rigid_transform_3D(A, B)


pcd_keypoints_l = o3d.geometry.PointCloud()
pcd_keypoints_l.points = o3d.utility.Vector3dVector(keypoints_l)

pcd_keypoints_l.rotate(R, center=(0, 0, 0))
pcd_keypoints_l.translate((t))

keypoints_l_tr = np.asarray(pcd_keypoints_l.points)

In [34]:
keypoints_l_tr

array([[-88.73018128,   0.87552646,   3.9319085 ],
       [-62.94033548,   1.83508429,  26.88120907],
       [-38.54293181, -11.00885654,  42.04497562],
       [-16.55831508, -22.38908235,  54.29344605],
       [  5.66166717, -33.80949364,  66.07100924],
       [ -7.419612  ,  -1.02650716,  20.68112199],
       [ 28.36644289,  -9.44211543,  20.0892651 ],
       [ 47.73088828, -14.69923462,  19.61387729],
       [ 65.66176963, -16.54997207,  19.87281518],
       [ -7.52006015,  -2.92862808,  -0.82125588],
       [ 29.79181368, -17.57373078,  -4.26524245],
       [ 51.40623584, -27.3962317 ,  -5.94021693],
       [ 69.40679806, -32.39748942,  -8.90370085],
       [-12.8796294 ,  -4.50539909, -21.13214236],
       [ 24.13672674, -13.64295261, -23.2835732 ],
       [ 45.68322511, -20.40163762, -24.99887669],
       [ 64.69960654, -21.82219099, -25.96846497],
       [-23.47261716,  -5.59729214, -38.42613224],
       [  6.79362163, -15.30944409, -40.90145552],
       [ 22.82200468, -21.75293